In [5]:
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from official_api.aliyun import face_compare

from AdvFaceGAN import Generator
from utils.dataset import Dataset

print("--------------------load test Dataset--------------------")
test_set = Dataset("C:/yy/datasets/lfw/lfw-aligned-112x112/", "target")
print("--------------------start batch queue--------------------")
batch_size = 8
test_set.start_batch_queue(
    batch_size=batch_size,
    batch_format="random_samples",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((112,112)),  # 调整图片大小
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 归一化
    ]),
    num_threads=3,
)
print("-------------------load Generator-----------------------")
fake_generator = Generator(is_target=True).eval().cuda()
model_generator_dict = torch.load(r"C:\yy\source\Desktop\读研是一条艰苦的道路\1. 论文\做实验\myCode\myCode\4 FrAdv\AdvFaceGAN\save_dir\target 4 8白盒 奇怪ssim 97ssim\model\00990_generator.pth", weights_only=True)
fake_generator.load_state_dict(model_generator_dict)

print("-----------------start api evaluation-------------------")
with torch.no_grad():
    epoch = 750
    after = []
    x=0
    for i in tqdm(range(0, epoch), total=epoch):
        batch = test_set.pop_batch_queue()
        source_faces = batch['sources'].cuda()
        target_faces = batch['targets'].cuda()
        perts, fake_afters = fake_generator.forward(source_faces, target_faces)
        for j in range(0, batch_size):
            torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './test2/target.png', nrow=1)
            torchvision.utils.save_image([fake_afters[j] * 0.5 + 0.5], './test2/fake.png', nrow=1)
            res = face_compare(face1_path='./test2/target.png', face2_path='./test2/fake.png')
            if res is None:
                x = x+1
                torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './wrong2/source{}_{}.png'.format(x, batch['sources_name'][j]), nrow=1)
                torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './wrong2/target{}_{}.png'.format(x, batch['targets_name'][j]), nrow=1)
            else:
                after.append(res)
    
    print(np.mean(after))
    after = np.array(after)
    print(np.sum(after > 61) / len(after))
    print(np.sum(after > 69) / len(after))
    print(np.sum(after > 75) / len(after))
    
'''
lfw:
同人 TAR
84.8268018281406
0.008348639171814994
0.01302387710803139
0.03623309400567708
非同人 FAR
4.331548709577778
1.0
1.0
1.0 
攻击
3 650 8白盒:
59.346987898603686
0.6284902190269186
0.30630329376358467
0.06554087945159673
0.9401924349466959

4 650 8白盒:
66.44565342554391
0.8005345806882727
0.5325760106916138
0.1944537253591714
0.9092922599315644

target 4 8白盒 奇怪ssim 97ssim
63.01459567335537
0.7244147157190636
0.40100334448160535
0.10518394648829431

celeba-hq:
攻击
3 700:
61.69760178467954
0.6906086663294554
0.40870005058168946
0.13791940650817738
0.953
4 650 8白盒无SE:
67.93522777581455
0.8257575757575758
0.6073232323232324
0.2967171717171717
0.9243021523952484
4 650 8白盒有SE
67.20090152697185
0.8108448928121059
0.5687263556116016
0.24968474148802017
0.926848818063736
'''

------------------------加载测试集-------------------------
13233 images of 5749 classes loaded
-----------------------启动分批队列------------------------


C:\Users\28769\AppData\Local\Temp\ipykernel_26192\2023120153.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_generator_dict = torch.load(r"C:\yy\source\Desktop\读研

-----------------------加载生成器------------------------
-----------------------开始api测试------------------------


  0%|          | 0/750 [00:00<?, ?it/s]C:\yy\installed_software\Anaconda3\envs\AdvFaceGAN\Lib\site-packages\torch\storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: B7DF2538-E478-5FC3-AD23-CB2D33360F56 Response: {'RequestId': 'B7DF2538-E478-5FC3-AD23-CB2D33360F56', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=B7DF2538-E478-5FC3-AD23-CB2D33360F56', 'statusCode': 404}


...

100%|█████████▉| 748/750 [1:38:35<00:16,  8.31s/it]

HTTPSConnectionPool(host='facebody.cn-shanghai.aliyuncs.com', port=443): Read timed out. (read timeout=10.0)


100%|██████████| 750/750 [1:38:59<00:00,  7.92s/it]

62.65134572926595
0.7191919191919192
0.40185185185185185
0.10858585858585859


'\nlfw:\n同人 TAR\n84.8268018281406\n0.008348639171814994\n0.01302387710803139\n0.03623309400567708\n非同人 FAR\n4.331548709577778\n1.0\n1.0\n1.0 \n攻击\n3 650 8白盒:\n59.346987898603686\n0.6284902190269186\n0.30630329376358467\n0.06554087945159673\n0.9401924349466959\n\n4 650 8白盒:\n66.44565342554391\n0.8005345806882727\n0.5325760106916138\n0.1944537253591714\n0.9092922599315644\n\n\nceleba-hq:\n攻击\n3 700:\n61.69760178467954\n0.6906086663294554\n0.40870005058168946\n0.13791940650817738\n0.953\n4 650 8白盒无SE:\n67.93522777581455\n0.8257575757575758\n0.6073232323232324\n0.2967171717171717\n0.9243021523952484\n4 650 8白盒有SE\n67.20090152697185\n0.8108448928121059\n0.5687263556116016\n0.24968474148802017\n0.926848818063736\n'

In [1]:
# calculate FAR
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from official_api.aliyun import face_compare

from utils.dataset import Dataset

print("-----------------------load test Dataset-------------------------")
test_set = Dataset("D:/datasets/lfw/lfw-aligned-112x112/", "target")
print("-----------------------start batch queue------------------------")
batch_size = 8
test_set.start_batch_queue(
    batch_size=batch_size,
    batch_format="random_samples",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((112,112)),  # 调整图片大小
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 归一化
    ]),
    num_threads=3,
    untarget_same=False
)

print("--------------------start api evaluation---------------------")
with torch.no_grad():
    epoch = 750
    before = []
    x=0
    for i in tqdm(range(0, epoch), total=epoch):
        batch = test_set.pop_batch_queue()
        source_faces = batch['sources'].cuda()
        target_faces = batch['targets'].cuda()
        for j in range(0, batch_size):
            torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './test/source.png', nrow=1)
            torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './test/target.png', nrow=1)
            res = face_compare(face1_path='./test/target.png', face2_path='./test/source.png')
            if res is None:
                x = x+1
                torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './wrong/source{}_{}.png'.format(x, batch['sources_name'][j]), nrow=1)
                torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './wrong/target{}_{}.png'.format(x, batch['targets_name'][j]), nrow=1)
            if res is not None:
                before.append(res)
    print(np.mean(before))
    before = np.array(before)
    print(np.sum(before < 61) / len(before))
    print(np.sum(before < 69) / len(before))
    print(np.sum(before < 75) / len(before))

------------------------加载测试集-------------------------
9164 images of 1680 classes loaded
-----------------------启动分批队列------------------------
-----------------------加载生成器------------------------
-----------------------开始api测试------------------------


  1%|          | 7/750 [00:53<1:33:45,  7.57s/it]

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: 87F6AB14-DD35-5D73-AAF3-762F0D50E24B Response: {'RequestId': '87F6AB14-DD35-5D73-AAF3-762F0D50E24B', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=87F6AB14-DD35-5D73-AAF3-762F0D50E24B', 'statusCode': 404}
./test/target.png  和  ./test/source.png


  7%|▋         | 56/750 [07:02<1:25:06,  7.36s/it]

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: CFABBCCD-1B2F-5460-91E1-993DC528CE51 Response: {'RequestId': 'CFABBCCD-1B2F-5460-91E1-993DC528CE51', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=CFABBCCD-1B2F-5460-91E1-993DC528CE51', 'statusCode': 404}
./test/target.png  和  ./test/source.png


...

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: E6E3200C-61A7-5D0B-8258-4FDF0D8E460A Response: {'RequestId': 'E6E3200C-61A7-5D0B-8258-4FDF0D8E460A', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=E6E3200C-61A7-5D0B-8258-4FDF0D8E460A', 'statusCode': 404}
./test/target.png  和  ./test/source.png


 96%|█████████▌| 717/750 [1:43:57<03:55,  7.13s/it]  

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: 46D62C30-0B28-5043-A4AB-125D2B1A96B7 Response: {'RequestId': '46D62C30-0B28-5043-A4AB-125D2B1A96B7', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=46D62C30-0B28-5043-A4AB-125D2B1A96B7', 'statusCode': 404}
./test/target.png  和  ./test/source.png


100%|██████████| 750/750 [1:47:55<00:00,  8.63s/it]

4.5995807754643065
0.9996659428762318
1.0
1.0


In [2]:
# calculate TAR
import numpy as np
import torch
import torchvision
from tqdm import tqdm
from official_api.aliyun import face_compare

from utils.dataset import Dataset

print("-----------------------load test Dataset-------------------------")
test_set = Dataset("D:/datasets/lfw/lfw-aligned-112x112/", "untarget")
print("-----------------------start batch queue------------------------")
batch_size = 8
test_set.start_batch_queue(
    batch_size=batch_size,
    batch_format="random_samples",
    transforms=torchvision.transforms.Compose([
        torchvision.transforms.Resize((112,112)),  # 调整图片大小
        torchvision.transforms.ToTensor(),  # 图片转为Tensor [0,1]
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),  # 归一化
    ]),
    num_threads=3,
    untarget_same=False
)

print("----------------------start api evaluation---------------------")
with torch.no_grad():
    epoch = 750
    before = []
    x=0
    for i in tqdm(range(0, epoch), total=epoch):
        batch = test_set.pop_batch_queue()
        source_faces = batch['sources'].cuda()
        target_faces = batch['targets'].cuda()
        for j in range(0, batch_size):
            torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './test/source.png', nrow=1)
            torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './test/target.png', nrow=1)
            res = face_compare(face1_path='./test/target.png', face2_path='./test/source.png')
            if res is None:
                x = x+1
                torchvision.utils.save_image([source_faces[j] * 0.5 + 0.5], './wrong/source{}_{}.png'.format(x, batch['sources_name'][j]), nrow=1)
                torchvision.utils.save_image([target_faces[j] * 0.5 + 0.5], './wrong/target{}_{}.png'.format(x, batch['targets_name'][j]), nrow=1)
            before.append(res)
    print(np.mean(before))
    before = np.array(before)
    print(np.sum(before < 61) / len(before))
    print(np.sum(before < 69) / len(before))
    print(np.sum(before < 75) / len(before))

------------------------加载测试集-------------------------
9164 images of 1680 classes loaded
-----------------------启动分批队列------------------------
-----------------------开始api测试------------------------


  6%|▌         | 44/750 [05:21<1:24:09,  7.15s/it]

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: 4F72C68B-23D9-52A5-90EB-FB001220DC7D Response: {'RequestId': '4F72C68B-23D9-52A5-90EB-FB001220DC7D', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=4F72C68B-23D9-52A5-90EB-FB001220DC7D', 'statusCode': 404}
./test/target.png  和  ./test/source.png


  7%|▋         | 51/750 [06:13<1:28:41,  7.61s/it]

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: 9D1483AB-1578-57E0-A388-60421E499217 Response: {'RequestId': '9D1483AB-1578-57E0-A388-60421E499217', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=9D1483AB-1578-57E0-A388-60421E499217', 'statusCode': 404}
./test/target.png  和  ./test/source.png


...

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: F64AFD4A-327B-5564-BC0C-69FC5C550CEF Response: {'RequestId': 'F64AFD4A-327B-5564-BC0C-69FC5C550CEF', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=F64AFD4A-327B-5564-BC0C-69FC5C550CEF', 'statusCode': 404}
./test/target.png  和  ./test/source.png


 91%|█████████ | 680/750 [1:23:05<08:38,  7.40s/it]

Error: InvalidImage.NotFoundFace code: 404, 图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST request id: 3F088FAD-B9A6-578F-84AE-DB41BB599A81 Response: {'RequestId': '3F088FAD-B9A6-578F-84AE-DB41BB599A81', 'HostId': 'facebody.cn-shanghai.aliyuncs.com', 'Code': 'InvalidImage.NotFoundFace', 'Message': '图像中没找到人脸，请检查您的图像中是否包含人脸或人脸太小 - 没有检测到人脸，可能是输入图片没有人脸、人脸太小(人脸占比应超过64*64像素)、图像中人脸质量较低或图像过度拉伸等导致。请检查是否存在人脸、人脸大小和图像人脸质量后再次尝试。 - EAS_FACE_NOT_EXIST', 'Recommend': 'https://api.aliyun.com/troubleshoot?q=InvalidImage.NotFoundFace&product=facebody&requestId=3F088FAD-B9A6-578F-84AE-DB41BB599A81', 'statusCode': 404}
./test/target.png  和  ./test/source.png


100%|██████████| 750/750 [1:31:38<00:00,  7.33s/it]

84.8268018281406
0.008348639171814994
0.01302387710803139
0.03623309400567708
